# Transformers utils for experiments

In [ ]:
#|default_exp hf.transformers.experiment

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export

from copy import deepcopy
from math import ceil
from typing import Any, Callable

import torch
from tqdm import tqdm
from datasets import Dataset
from transformers import TrainingArguments, pipeline
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

from bellek.hf.transformers.utils import load_tokenizer_model
from bellek.hf.datasets.utils import load_datasets
from bellek.logging import get_logger
from bellek.utils import NestedDict, generate_time_id

log = get_logger(__name__)

In [ ]:
#|export

def prepare_config_for_fp(config: NestedDict):
    if not torch.cuda.is_available():
        return config

    # Set float precision
    if config.at("pretrained_model.torch_dtype") in {"float16", "bfloat16"}:
        major, _ = torch.cuda.get_device_capability()
        gpu_supports_bf = major >= 8
        if gpu_supports_bf:
            log.info("GPU supports bfloat16.")
        else:
            log.info("GPU does not support bfloat16.")
        
        if config.at("pretrained_model.torch_dtype") == "bfloat16" and gpu_supports_bf:
            log.info("Using bfloat16.")
            torch_dtype, bf16, fp16, bnb_4bit_compute_dtype = ("bfloat16", True, False, "bfloat16")
        else:
            log.info("Using float16.")
            torch_dtype, bf16, fp16, bnb_4bit_compute_dtype = ("float16", False, True, "float16")
    else:
        log.info("Not using half-precision float.")
        torch_dtype, bf16, fp16, bnb_4bit_compute_dtype = (None, None, None, None)

    if config.at("pretrained_model.torch_dtype"):
        config.set("pretrained_model.torch_dtype", torch_dtype)
    if config.at("pretrained_model.quantization_config.load_in_4bit"):
        config.set("pretrained_model.quantization_config.bnb_4bit_compute_dtype", bnb_4bit_compute_dtype)
    if config.at("trainer.training_args.bf16") or config.at("trainer.training_args.fp16"):
        config.set("trainer.training_args.bf16", bf16)
        config.set("trainer.training_args.fp16", fp16)
    
    return config

def preprocess_config(config: NestedDict):
    config = deepcopy(config)

    if isinstance(config.at("dataset.train"), dict):
        config.set("dataset.train", [config.at("dataset.train")])
    if isinstance(config.at("dataset.validation"), dict):
        config.set("dataset.validation", [config.at("dataset.validation")])

    config = prepare_config_for_fp(config)
    
    # Generate unique model id for output model
    if (out_model_id := config.at("hfhub.model_id")) and config.at("metaconfig.preprocessing.unique_hfhub_model_id"):
        if "-peft" not in out_model_id and config.at("trainer.lora"):
            out_model_id += "-peft"
        if wandb_run_id := config.at("wandb.run_id"):
            out_model_id += f"-{wandb_run_id}"
        else:
            out_model_id += f"-{generate_time_id()}"
        config.set("hfhub.model_id", out_model_id)

    return config


In [ ]:
#|hide

import json
config = json.loads("""
{
  "seed": 42,
  "dataset": {
    "train": {
      "path": "bdsaglam/webnlg-jerx-sft-multi-turn-multi-sentence-openai",
      "split": "train"
    },
    "validation": {
      "path": "bdsaglam/webnlg-jerx-sft-multi-turn-multi-sentence-openai",
      "split": "dev[:256]"
    }
  },
  "pretrained_model": {
    "model_name_or_path": "meta-llama/llama-2-7b-chat-hf",
    "torch_dtype": "bfloat16",
    "quantization_config": {
      "load_in_8bit": false,
      "load_in_4bit": true,
      "bnb_4bit_quant_type": "nf4"
    }
  },
  "trainer": {
    "packing": false,
    "lora": {
      "lora_alpha": 16,
      "lora_dropout": 0.1,
      "r": 64,
      "bias": "none",
      "task_type": "CAUSAL_LM"
    },
    "response_template": "[/INST]",
    "response_template_context": " ",
    "training_args": {
      "bf16": true,
      "fp16": false,
      "group_by_length": true,
      "per_device_train_batch_size": 4,
      "gradient_accumulation_steps": 2,
      "gradient_checkpointing": true,
      "max_grad_norm": 0.3,
      "weight_decay": 0.001,
      "learning_rate": 0.0002,
      "lr_scheduler_type": "cosine",
      "warmup_ratio": 0.03,
      "optim": "paged_adamw_32bit",
      "max_steps": -1,
      "num_train_epochs": 1,
      "logging_steps": 25,
      "save_steps": 0,
      "report_to": "wandb"
    }
  },
  "evaluation": {
    "pipeline": {
      "batch_size": 8
    },
    "metric": "bdsaglam/jer"
  },
  "wandb": {
    "mode": "online",
    "entity": "bdsaglam",
    "project": "thesis-kgcons"
  },
  "hfhub": {
    "model_id": "bdsaglam/llama-2-7b-chat-jerx"
  },
  "metaconfig": {
    "preprocessing": {
      "resolve_paths": false,
      "unique_hfhub_model_id": true
    }
  }
}
""")

preprocess_config(NestedDict(config))

{'seed': 42,
 'dataset': {'train': [{'path': 'bdsaglam/webnlg-jerx-sft-multi-turn-multi-sentence-openai',
    'split': 'train'}],
  'validation': [{'path': 'bdsaglam/webnlg-jerx-sft-multi-turn-multi-sentence-openai',
    'split': 'dev[:256]'}]},
 'pretrained_model': {'model_name_or_path': 'NousResearch/Llama-2-7b-chat-hf',
  'torch_dtype': 'bfloat16',
  'quantization_config': {'load_in_8bit': False,
   'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4'}},
 'trainer': {'packing': False,
  'lora': {'lora_alpha': 16,
   'lora_dropout': 0.1,
   'r': 64,
   'bias': 'none',
   'task_type': 'CAUSAL_LM'},
  'response_template': '[/INST]',
  'response_template_context': ' ',
  'training_args': {'bf16': True,
   'fp16': False,
   'group_by_length': True,
   'per_device_train_batch_size': 4,
   'gradient_accumulation_steps': 2,
   'gradient_checkpointing': True,
   'max_grad_norm': 0.3,
   'weight_decay': 0.001,
   'learning_rate': 0.0002,
   'lr_scheduler_type': 'cosine',
   'warmup_ratio': 0

In [ ]:
#|export

def make_datacollator(tokenizer, response_template: str | None, response_template_context: str | None = None):
    if not response_template:
        return None

    if response_template_context is None:
        log.info(f"Creating completion-only data collator with response template '{response_template}'")
        data_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
    else:
        log.info(f"Creating completion-only data collator with response template '{response_template}' and context '{response_template_context}'")
        response_template_with_context = response_template_context + response_template
        response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[len(response_template_context):] 
        data_collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)
    
    return data_collator

In [ ]:
#|export

def prepare_model_for_training(tokenizer, model):
    model_id = model.name_or_path.lower()

    if "llama-2" in model_id:
        from bellek.hf.transformers.llama2 import prepare_llama2_for_training

        log.info("Base model is a llama-2 model, preparing it for training.")
        prepare_llama2_for_training(tokenizer, model)
    
    elif "llama-3" in model_id:
        from bellek.hf.transformers.llama3 import prepare_llama3_for_training

        log.info("Base model is a llama-3 model, preparing it for training.")
        prepare_llama3_for_training(tokenizer, model)
    
    else:
        log.warning(f"Base model '{model_id}' is not a llama-2 or llama-3 model, no special preparation is done.")

In [ ]:
#|export

def calculate_token_counts(tokenizer, dataset: Dataset, dataset_text_field: str | None):
    if dataset_text_field is None:
        if "messages" not in dataset.column_names:
            raise ValueError("Dataset must have 'messages' columns if `dataset_text_field` is not specified.")
        
        dataset_text_field = "text"
        dataset = dataset.map(
            lambda example: {dataset_text_field: tokenizer.apply_chat_template(example["messages"], tokenize=False, add_generation_prompt=False)}
        )
    
    # Inspect token counts
    tokenized_train_ds = dataset.map(lambda examples: tokenizer(examples[dataset_text_field]), batched=True)
    token_counts = [len(input_ids) for input_ids in tokenized_train_ds["input_ids"]]
    log.info(f"Input token counts: min={min(token_counts)}, max={max(token_counts)}")
    return token_counts

In [ ]:
#|export

def fine_tune(config: NestedDict):
    from peft import LoraConfig

    # Base model
    pretrained_model_config = config["pretrained_model"]
    model_id = pretrained_model_config.pop("model_name_or_path")
    tokenizer, base_model = load_tokenizer_model(model_id, **pretrained_model_config)
    log.info(f"Loaded base model {model_id}")

    # Prepare model for training
    prepare_model_for_training(tokenizer, base_model)

    # Train dataset
    train_ds = load_datasets(config.at("dataset.train")).shuffle(seed=config.at("seed"))
    log.info(f"Loaded training dataset with {len(train_ds)} samples.")

    # Inspect token counts
    dataset_text_field = config.at("trainer.dataset_text_field")
    token_counts = calculate_token_counts(tokenizer, train_ds, dataset_text_field)
    log.info(f"Input token counts: min={min(token_counts)}, max={max(token_counts)}")

    # Supervised fine-tuning
    if config.at("trainer.max_seq_length") is None:
        config.set("trainer.max_seq_length", ceil(max(token_counts) / 8) * 8)
    max_seq_length = config.at("trainer.max_seq_length")
    log.info(f"Setting max_seq_length={max_seq_length}")

    peft_config = LoraConfig(**config.at("trainer.lora", {}))

    packing = config.at("trainer.packing", False)

    data_collator = make_datacollator(
        tokenizer,
        config.at("trainer.response_template"),
        config.at("trainer.response_template_context"),
    )

    training_args = TrainingArguments(
        output_dir="./results",
        **config.at("trainer.training_args"),
    )

    trainer = SFTTrainer(
        tokenizer=tokenizer,
        model=base_model,
        peft_config=peft_config,
        train_dataset=train_ds,
        dataset_text_field=dataset_text_field,
        max_seq_length=max_seq_length,
        packing=packing,
        data_collator=data_collator,
        args=training_args,
    )
    log.info("Starting training...")
    trainer.train()

    # Save trained model
    log.info("Saving model...")
    final_model_id = config.at("hfhub.model_id")
    trainer.model.push_to_hub(final_model_id)
    tokenizer.push_to_hub(final_model_id)
    log.info(f"Uploaded PEFT adapters to HF Hub as {final_model_id}")

    return trainer


In [ ]:
#|export

def prepare_model_for_inference(tokenizer, model):
    model_id = model.name_or_path.lower()

    if "llama-2" in model_id:
        from bellek.hf.transformers.llama2 import prepare_llama2_for_inference

        log.info("Base model is a llama-2 model, preparing it for inference.")
        prepare_llama2_for_inference(tokenizer, model)
    
    elif "llama-3" in model_id:
        from bellek.hf.transformers.llama3 import prepare_llama3_for_inference

        log.info("Base model is a llama-3 model, preparing it for inference.")
        prepare_llama3_for_inference(tokenizer, model)
    
    else:
        log.warning(f"Base model '{model_id}' is not a llama-2 or llama-3 model, no special preparation is done.")

In [ ]:
# |export


def _load_tokenizer_model(config: NestedDict):
    model_id = config.at("hfhub.model_id")
    kwargs = deepcopy(config.get("pretrained_model", {}))
    kwargs.pop("model_name_or_path", None)
    return load_tokenizer_model(model_id, **kwargs)


def make_pipeline(config, tokenizer, model):
    prepare_model_for_inference(tokenizer, model)
    return pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        **config.at("evaluation.pipeline", {}),
    )


def flat_pipeline(pipe):
    def func(inputs, **kwargs) -> list[str]:
        return [result[0]["generated_text"] for result in tqdm(pipe(inputs, **kwargs))]

    return func


def predict(
    pipe,
    dataset,
    *,
    output_parse_fn: Callable[[str], Any] | None = None,
    **generation_kwargs,
):
    output_parse_fn = output_parse_fn or (lambda x: x)

    # Setup generation parameters
    generation_kwargs["return_full_text"] = False

    if "max_new_tokens" not in generation_kwargs:
        tokenized_outputs = dataset.map(lambda examples: pipe.tokenizer(examples["output"]), batched=True)
        token_counts = [len(input_ids) for input_ids in tokenized_outputs["input_ids"]]
        log.info(f"Output token counts: min={min(token_counts)}, max={max(token_counts)}")
        generation_kwargs["max_new_tokens"] = ceil(max(token_counts) / 8) * 8

    terminators = generation_kwargs.pop("terminators", [])
    eos_token_ids = {pipe.tokenizer.eos_token_id}
    for terminator in terminators:
        if isinstance(terminator, int):
            eos_token_ids.add(terminator)
        elif isinstance(terminator, str):
            eos_token_ids.add(pipe.tokenizer.convert_tokens_to_ids(terminator))
        else:
            raise ValueError(f"Invalid terminator token {terminator}.")
    generation_kwargs["eos_token_id"] = sorted(eos_token_ids)

    # Generate text
    log.info(f"Running pipeline on dataset with {len(dataset)} samples...")
    generations = flat_pipeline(pipe)(dataset["input"], **generation_kwargs)

    # Parse outputs
    predictions = [output_parse_fn(text) for text in generations]
    references = [output_parse_fn(text) for text in dataset["output"]]

    # Create dataframe
    dataf = dataset.to_pandas()
    dataf["generation"] = generations
    dataf["prediction"] = predictions
    dataf["reference"] = references

    return dataf


def evaluate_(
    config,
    *,
    tokenizer=None,
    model=None,
    metric_kwargs: dict | None = None,
    output_parse_fn: Callable[[str], Any] | None = None,
):
    import evaluate

    # Load validation dataset
    dataset_config = config.at("dataset.validation")
    assert dataset_config, "Validation dataset is not provided!"
    dataset = load_datasets(dataset_config)
    assert len(dataset) > 0, "Validation dataset is empty!"
    
    # Ensure the dataset has input/output columns
    cols = dataset[0].keys()
    if "input" not in cols or "output" not in cols:
        if "messages" not in dataset.column_names:
            raise ValueError("Dataset must have 'messages' column if 'input' and 'output' columns are not provided.")
        dataset = dataset.map(
            lambda x: {"input": x["messages"][:-1], "output": x["messages"][-1]["content"]}
        ).remove_columns("messages")

    # Prepare text generation pipeline
    if tokenizer is None or model is None:
        tokenizer, model = _load_tokenizer_model(config)

    pipe = make_pipeline(config, tokenizer, model)

    dataf = predict(
        pipe,
        dataset,
        output_parse_fn=output_parse_fn,
        **config.at("evaluation.generation_params", {}),
    )

    # Compute metrics
    metric = evaluate.load(config.at("evaluation.metric"))
    metric_kwargs = metric_kwargs or {}
    scores = metric.compute(predictions=dataf["prediction"].values, references=dataf["reference"].values, **metric_kwargs)

    return scores, dataf

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()